In [1]:
# The goal of this project was to create a methodology for determining if a given invoice in a given month was offered at a promotional price.
# The data used already contains a verified promo flag, making it ideal for testing new methodologies for their accuracy.

# Part 1: We'll show the process for creating the most accurate promo flag through several iterative steps.
# Part 2: We'll show the method that works sufficiently well, and can be implemented at scale with 2 major steps.

In [2]:
# First, we need to import some libraries for handling the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# We import our data from CSV and create a dataframe
df = pd.read_csv('Purchase Orders - For Git.csv')
df['Month'] = pd.to_datetime(df['Month'], infer_datetime_format=True)
df = df.sort_values(by='Month')
df

,Month,Product,Customer,Cases,Unit Price,Promo,Total Price
0,2019-02-01,Product 1,Customer 1,90,15.6,0,1404.0
1,2019-02-01,Product 2,Customer 1,90,15.6,0,1404.0
2,2019-02-01,Product 3,Customer 1,90,15.6,0,1404.0
3,2019-02-01,Product 4,Customer 1,72,15.6,0,1123.2
4,2019-02-01,Product 5,Customer 1,72,15.6,0,1123.2
...,...,...,...,...,...,...,...
2455,2023-05-01,Product 6,Customer 14,60,19.2,0,1152.0
2454,2023-05-01,Product 1,Customer 14,120,19.2,0,2304.0
2453,2023-05-01,Product 8,Customer 15,24,19.2,0,460.8
2451,2023-05-01,Product 1,Customer 15,72,19.2,0,1382.4


In [4]:
# Get the monthly average price for each product
avg_df = df.groupby(['Product', 'Month'], as_index=False).agg({'Cases': np.sum, 'Total Price': np.sum})
avg_df['Avg. Monthly Unit Price'] = avg_df['Total Price']/avg_df['Cases']

# Join the two dataframes
new_df = pd.merge(df, avg_df, on=['Product','Month'], how='outer', suffixes=('','_x'))
new_df = new_df.rename(columns={'Unit Price_x': 'Avg. Monthly Unit Price'})

#Add new column that takes the previous period's unit price for each customer/product combo
new_df['Previous Price'] = new_df.groupby(['Product','Customer'])['Unit Price'].shift(1)
new_df['Previous Avg. Price'] = new_df.groupby(['Product'])['Avg. Monthly Unit Price'].shift(1)

new_df

,Month,Product,Customer,Cases,Unit Price,Promo,Total Price,Cases_x,Total Price_x,Avg. Monthly Unit Price,Previous Price,Previous Avg. Price
0,2019-02-01,Product 1,Customer 1,90,15.6,0,1404.0,90,1404.0,15.6,NaN,NaN
1,2019-02-01,Product 2,Customer 1,90,15.6,0,1404.0,90,1404.0,15.6,NaN,NaN
2,2019-02-01,Product 3,Customer 1,90,15.6,0,1404.0,90,1404.0,15.6,NaN,NaN
3,2019-02-01,Product 4,Customer 1,72,15.6,0,1123.2,72,1123.2,15.6,NaN,NaN
4,2019-02-01,Product 5,Customer 1,72,15.6,0,1123.2,72,1123.2,15.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2504,2023-05-01,Product 3,Customer 10,90,19.2,0,1728.0,1326,25459.2,19.2,16.32,19.20
2505,2023-05-01,Product 3,Customer 5,60,19.2,0,1152.0,1326,25459.2,19.2,19.20,19.20
2506,2023-05-01,Product 8,Customer 20,18,19.2,0,345.6,42,806.4,19.2,16.32,19.20
2507,2023-05-01,Product 8,Customer 15,24,19.2,0,460.8,42,806.4,19.2,19.20,19.20


In [5]:
#Compare current price to previous period's price. If lower, flag as promo.
new_df['Promo Flag 1'] = np.where(new_df['Unit Price'] < new_df['Previous Price'],1,0)

#Check Promo Flag 1 to true Promo Flag and get the total number of rows where they don't match
new_df['Check 1'] = abs(new_df['Promo Flag 1'] - new_df['Promo'])
print(str(sum(new_df['Check 1'])) + ' rows (' + str(round(sum(new_df['Check 1']/len(new_df['Check 1']))*100, 2)) + '%) do not match true promo flag')

722 rows (28.78%) do not match true promo flag


In [6]:
#Refine promo flag by checking if the promo went on for more than 1 period
new_df['Promo Flag 2'] = np.where(
    new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(1), 1,
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(2), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(3), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(4), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(5), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(6), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(7), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(8), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(9), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(10), 1, 
    np.where(new_df['Unit Price'] < new_df.groupby(['Product','Customer'])['Unit Price'].shift(11), 1, 0)))))))))))

new_df['Check 2'] = abs(new_df['Promo Flag 2'] - new_df['Promo'])
print(str(sum(new_df['Check 2'])) + ' rows (' + str(round(sum(new_df['Check 2']/len(new_df['Check 2']))*100, 2)) + '%) do not match true promo flag')

82 rows (3.27%) do not match true promo flag


In [7]:
#Refine promo flag by checking against average monthly unit price
new_df['Promo Flag 3'] = np.where(
    new_df['Promo Flag 2'] == 0, 
        np.where(new_df['Unit Price'] < new_df['Avg. Monthly Unit Price'],
            np.where(abs(new_df['Unit Price'] - new_df['Avg. Monthly Unit Price'])/new_df['Unit Price'] >= 0.05, 1, 0), 0) ,new_df['Promo Flag 2'])

new_df['Check 3'] = abs(new_df['Promo Flag 3'] - new_df['Promo'])
print(str(sum(new_df['Check 3'])) + ' rows (' + str(round(sum(new_df['Check 3']/len(new_df['Check 3']))*100, 2)) + '%) do not match true promo flag')

65 rows (2.59%) do not match true promo flag


In [8]:
#Account for first purchase of a product ever after all of the other checks, because we can't know if it is promo in the first purchase ever.
new_df['Promo Flag 4'] = np.where(
    new_df['Promo Flag 3'] == 0,
        np.where(np.isnan(new_df['Previous Price']) == True, new_df['Previous Price'], 0), new_df['Promo Flag 3'])

new_df['Check 4'] = abs(new_df['Promo Flag 4'] - new_df['Promo'])
print(str(np.nansum(new_df['Check 4'])) + ' rows (' + str(round(np.nansum(new_df['Check 4']/len(new_df['Check 4']))*100, 2)) + '%) do not match true promo flag')

34.0 rows (1.36%) do not match true promo flag


In [9]:
#Create a table with the predicted promo price for a given month based on earlier flag
max_price_df = new_df.groupby(['Month','Product'], as_index=False).agg({'Unit Price': np.average, 'Promo Flag 4': np.max})
max_price_df['Predicted Promo Price (Avg)'] = np.where(
    max_price_df['Promo Flag 4'] == 1, max_price_df['Unit Price'], -99)

# Merge the predicted promo price with the dataframe with our other promo flags
final_df = pd.merge(new_df, max_price_df, on=['Month','Product'], how='outer')

# Create a final flag that accounts for other promo prices detected in the same month
final_df['Final Flag'] = np.where(
    final_df['Promo Flag 4_x'] == 0,
        np.where(final_df['Unit Price_x'] <= final_df['Predicted Promo Price (Avg)'], 1, 0), final_df['Promo Flag 4_x'])

final_df['Check 5'] = abs(final_df['Final Flag'] - final_df['Promo'])
print(str(np.nansum(final_df['Check 5'])) + ' rows (' + str(round(np.nansum(final_df['Check 5']/len(final_df['Check 5']))*100, 2)) + '%) do not match true promo flag')

9.0 rows (0.36%) do not match true promo flag


In [10]:
# We were able to get a promo flag that differed from the true promo flag in only 9 cases out of 2509 cases (0.36% of the cases)
# However, we may not need this level of accuracy, especially for a production model to work at scale
# Now we will show the final option chosen to have a flag that is accurate enough and simple to implement

In [11]:
# This section creates a dictionary with the true price for all month, customer, and product combinations
# Group data to get max price for each product, customer, and month
new_df2 = df.groupby(['Product','Customer','Month']).agg({'Unit Price': np.max})

# Create lag values for each month and get max of all shifts and current month to get "True Price"
shift_df = new_df2
shift_df['True Price'] = shift_df.groupby(['Product','Customer'])['Unit Price'].cummax()

#Create True Price Dictionary
true_price_dict = shift_df.to_dict()

#Convert dictionary to dataframe for True Price
true_price_df = pd.DataFrame.from_dict(true_price_dict).reset_index().rename(columns={'level_0': 'Product','level_1': 'Customer', 'level_2': 'Month'})

#Merge True Price dataframe to raw data
merged_df = pd.merge(df, true_price_df, on=['Product','Customer','Month'], how='outer', suffixes=('','_x'))
merged_df

,Month,Product,Customer,Cases,Unit Price,Promo,Total Price,Unit Price_x,True Price
0,2019-02-01,Product 1,Customer 1,90,15.6,0,1404.0,15.6,15.6
1,2019-02-01,Product 2,Customer 1,90,15.6,0,1404.0,15.6,15.6
2,2019-02-01,Product 3,Customer 1,90,15.6,0,1404.0,15.6,15.6
3,2019-02-01,Product 4,Customer 1,72,15.6,0,1123.2,15.6,15.6
4,2019-02-01,Product 5,Customer 1,72,15.6,0,1123.2,15.6,15.6
...,...,...,...,...,...,...,...,...,...
2504,2023-05-01,Product 5,Customer 5,60,19.2,0,1152.0,19.2,19.2
2505,2023-05-01,Product 6,Customer 14,60,19.2,0,1152.0,19.2,19.2
2506,2023-05-01,Product 1,Customer 14,120,19.2,0,2304.0,19.2,19.2
2507,2023-05-01,Product 8,Customer 15,24,19.2,0,460.8,19.2,19.2


In [12]:
# This section creates a promo flag where the Unit Price is less than the True Price, and it shows how many rows are incorrect compared to the true promo flag
merged_df['Promo Flag TP'] = np.where(
    merged_df['Unit Price'] < merged_df['True Price'],1,0)

merged_df['Check TP'] = abs(merged_df['Promo Flag TP'] - merged_df['Promo'])
print('Promo Flag is Incorrect ' + str(sum(merged_df['Check TP'])) + ' times out of ' + str(len(merged_df['Promo'])) + ' rows (' + str(round(100*sum(merged_df['Check TP'])/len(merged_df['Promo']),2)) + '%)')
print('True Promo Rate: ' + str(round(100*sum(merged_df['Promo'])/len(merged_df['Promo']),2)) + '%')
print('Promo Rate from True Price Method: ' + str(round(100*sum(merged_df['Promo Flag TP'])/len(merged_df['Promo']),2)) + '%')

Promo Flag is Incorrect 72 times out of 2509 rows (2.87%)
True Promo Rate: 52.21%
Promo Rate from True Price Method: 49.58%
